In [1]:
import pandas as pd
import numpy as np
from mealpy.optimizer import Optimizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time            
import numpy as np

from mealpy.utils import io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming you have a dataset 'X' and corresponding labels 'y'

datasetPoly = pd.read_csv('labelled-balanced-data11.csv')
X, y = datasetPoly.iloc[:, :-1], datasetPoly.iloc[:, -1]

# Determine the number of features
num_features = X.shape[1]




In [2]:
class NomadZOA(Optimizer):
    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False
        
    def evolve(self, epoch):
        
        # PHASE1: Foraging Behaviour
        pop_new = []
        for idx in range(0, self.pop_size):
            r1 = np.round(1 + np.random.rand())
            pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.g_best[self.ID_POS] - r1 * self.pop[idx][self.ID_POS])   # Eq. 3
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)
            
        # PHASE2: Defense strategies against predators
        kk = np.random.permutation(self.pop_size)[0]
        pop_new = []
        while True:
            if np.random.rand() < 0.5:
                # S1: the lion attacks the zebra and thus the zebra chooses an escape strategy
                r2 = 0.1
                pos_new = self.pop[idx][self.ID_POS] + r2 * (2 + np.random.rand(self.problem.n_dims) - 1) * (1 - (epoch+1)/self.epoch) * self.pop[idx][self.ID_POS]
            else:
                # S2: other predators attack the zebra and the zebra will choose the offensive strategy
                r2 = np.random.randint(1, 3)
                pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.pop[kk][self.ID_POS] - r2 * self.pop[idx][self.ID_POS])

            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)

            # Check if the newly generated pos_new is different from the current position
            if not np.allclose(pos_new, self.pop[idx][self.ID_POS]):
                break  # Exit the loop if a different position is found


In [4]:


# Define a fitness function that evaluates the performance of Random Forest on the selected features
def fitness_function(solution):
    selected_features = [i for i, selected in enumerate(solution) if selected]
    if not selected_features:
        return 0.0  # No features selected, return fitness value of 0
    
    X_subset = X.iloc[:, selected_features]  # Select the corresponding columns from the dataset

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=42)

    # Create a Random Forest classifier
    rfc = RandomForestClassifier()

    # Train the classifier
    rfc.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = rfc.predict(X_test)

    # Calculate the accuracy score as the fitness value
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Number of features
num_features = len(X.columns)

problemNomad = {
    "fit_func": fitness_function,
    "lb": [0] * num_features,  # Lower bound for each feature (0 means not selected)
    "ub": [1] * num_features,  # Upper bound for each feature (1 means selected)
    "minmax": "max", # Maximizing the fitness function
    "save_population": True
}

# Number of optimization runs
num_runs = 10

# Store results
all_results_nzoa = []

for run in range(num_runs):
    # Create the PoliticalZOA optimization model
    modelnomad = NomadZOA(epoch=100, pop_size=100)

    # Measure the total execution time
    start_time = time.time()

    # Solve the optimization problem
    best_position, best_fitness = modelnomad.solve(problemNomad)

    # Calculate and print the total execution time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Run {run + 1}: Total execution time: {total_time} seconds")

    # Create a dictionary to map feature names to their corresponding position values
    feature_position_dict = {feature_name: position for feature_name, position in zip(X.columns, best_position)}

    # Store results
    all_results_nzoa.append({"position_values": feature_position_dict, "fitness": best_fitness})



2023/12/28 08:55:29 AM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2023/12/28 08:58:50 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8616803278688525, Global best: 0.8616803278688525, Runtime: 97.22233 seconds
2023/12/28 09:00:24 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8616803278688525, Global best: 0.8616803278688525, Runtime: 93.95734 seconds
2023/12/28 09:02:01 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8616803278688525, Global best: 0.8616803278688525, Runtime: 97.42088 seconds
2023/12/28 09:03:37 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8616803278688525, Global best: 0.8616803278688525, Runtime: 96.06454 seconds
2023/12/28 09:05:32 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8616803278688525, Global best: 0.8616803278688525, Runtime: 114.73690 seconds
2023/12/28 09:07:51 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Curren

2023/12/28 10:28:37 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 50, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 108.58689 seconds
2023/12/28 10:30:24 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 51, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 106.44210 seconds
2023/12/28 10:32:45 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 52, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 141.69013 seconds
2023/12/28 10:35:29 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 53, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 164.19075 seconds
2023/12/28 10:38:12 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 54, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 163.07231 seconds
2023/12/28 10:40:47 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 55, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 154.83467 second

2023/12/28 12:04:29 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 100, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 91.39180 seconds


Run 1: Total execution time: 11342.80670595169 seconds


2023/12/28 12:04:30 PM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2023/12/28 12:07:53 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 99.49417 seconds
2023/12/28 12:09:32 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 99.37590 seconds
2023/12/28 12:11:11 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 98.74462 seconds
2023/12/28 12:12:52 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 101.09002 seconds
2023/12/28 12:14:35 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 102.71518 seconds
2023/12/28 12:16:18 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Curre

...

Run 9: Total execution time: 9475.176898956299 seconds


2023/12/29 09:38:42 AM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2023/12/29 09:42:00 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 95.56571 seconds
2023/12/29 09:43:37 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 97.02660 seconds
2023/12/29 09:45:15 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 97.89678 seconds
2023/12/29 09:46:53 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 97.75777 seconds
2023/12/29 09:48:31 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 97.96791 seconds
2023/12/29 09:50:10 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Current

2023/12/29 11:10:01 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 50, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 122.29535 seconds
2023/12/29 11:12:12 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 51, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 131.16405 seconds
2023/12/29 11:14:08 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 52, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 115.77216 seconds
2023/12/29 11:16:50 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 53, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 161.90916 seconds
2023/12/29 11:18:36 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 54, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 106.77739 seconds
2023/12/29 11:21:06 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 55, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 149.99355 second

2023/12/29 12:49:16 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 100, Current best: 0.8647540983606558, Global best: 0.8647540983606558, Runtime: 103.47785 seconds


Run 10: Total execution time: 11434.757798671722 seconds


In [5]:
# Print the feature names and corresponding position values for each run in sorted order
for run, result in enumerate(all_results_nzoa, start=1):
    sorted_features = sorted(result["position_values"].items(), key=lambda x: x[1])
    print(f"\nRun {run}: Fitness (Accuracy): {result['fitness']}")
    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")


Run 1: Fitness (Accuracy): 0.8637295081967213
Feature: Diet, Position Value: 0.01880352867788928
Feature: Region, Position Value: 0.09706584596670872
Feature: BMI, Position Value: 0.10131835997810712
Feature: Gender, Position Value: 0.18142689504341183
Feature: Age, Position Value: 0.2018959850242153
Feature: Waist_Size, Position Value: 0.20683862354608
Feature: Family_History, Position Value: 0.2447830199344288
Feature: Smoking, Position Value: 0.34609935183784946
Feature: BP, Position Value: 0.37404142091426584
Feature: Physical_Activity, Position Value: 0.6268001755879528

Run 2: Fitness (Accuracy): 0.8642418032786885
Feature: Smoking, Position Value: 0.07484724536353722
Feature: Age, Position Value: 0.101013150565427
Feature: Gender, Position Value: 0.1523431087342741
Feature: Diet, Position Value: 0.1659092266127682
Feature: Waist_Size, Position Value: 0.1697440125672415
Feature: Physical_Activity, Position Value: 0.21627291237838792
Feature: BP, Position Value: 0.242287623624282

In [9]:
# Save the sorted DataFrame to a CSV file
results_df_sorted.to_csv('sorted_results nomad.csv', index=False)

# Print the feature names and corresponding position values for each run
for rank, (_, result) in enumerate(results_df_sorted.iterrows(), start=1):
    print(f"\nRank {rank}: Fitness (Accuracy): {result['fitness']}")

    # Sort feature names by their position values
    sorted_features = sorted(result['position_values'].items(), key=lambda x: x[1], reverse=True)

    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")



Rank 1: Fitness (Accuracy): 0.8652663934426229
Feature: Physical_Activity, Position Value: 0.8834878291189873
Feature: Smoking, Position Value: 0.8277363966952584
Feature: Gender, Position Value: 0.7307491715228936
Feature: Family_History, Position Value: 0.6800336380783417
Feature: Age, Position Value: 0.5963897255852021
Feature: BP, Position Value: 0.5232355166492486
Feature: Waist_Size, Position Value: 0.4809425736468643
Feature: Diet, Position Value: 0.3921485505253223
Feature: Region, Position Value: 0.27062890448343035
Feature: BMI, Position Value: 0.19464596259885103

Rank 2: Fitness (Accuracy): 0.8647540983606558
Feature: Smoking, Position Value: 0.795399925737113
Feature: Gender, Position Value: 0.6568226877916293
Feature: BMI, Position Value: 0.6252289465135746
Feature: Family_History, Position Value: 0.4642327660954487
Feature: BP, Position Value: 0.4439758821938591
Feature: Physical_Activity, Position Value: 0.35265886126782886
Feature: Diet, Position Value: 0.344851288366